# Lab-03 Neural Networks

## Loading Data

In [1]:
import os
import re
import cv2

images = []
pose_name = []
images_path = []
images_pixels = []
labels = []

dict = {}

i=0
dataPath = './yogaData'

for directory in os.listdir(dataPath):
    dirPath = os.path.join(dataPath, directory)
    if os.path.isdir(dirPath):
        pose_name.append(directory)
        for img in os.listdir(dirPath):
            if len(re.findall('.png',img.lower())) != 0 or len(re.findall('.jpg',img.lower())) != 0 or len(re.findall('.jpeg',img.lower())) != 0:
                img_path = os.path.join(dirPath,img)
                images.append(img)
                images_path.append(img_path)
                img_pix = cv2.imread(img_path,1)
                images_pixels.append(cv2.resize(img_pix, (100,100)))
                labels.append(i)

    i = i+1

print("Total images: ", len(images))
print("Total images path: ", len(images_path))
print("Total postures: ", len(pose_name))
print("Total images_pixels: ", len(images_pixels))

libpng warning: iCCP: known incorrect sRGB profile


Total images:  2177
Total images path:  2177
Total postures:  9
Total images_pixels:  2177


In [7]:
import random

shuffled = list(zip(images_pixels,labels))
random.shuffle(shuffled)

train_data, labels_data = zip(*shuffled)

In [26]:
from tensorflow.keras.utils import to_categorical
import numpy as np

X_data = np.array(train_data)/255
Y_data =  to_categorical(labels_data, num_classes = 10) # Should be 9 classes
sum(Y_data)

array([210., 268.,   0., 197., 209., 274., 198., 226., 334., 261.],
      dtype=float32)

In [27]:
sum(Y_data)
Y_data = np.delete(Y_data, 2, 1)

In [29]:
sum(sum(Y_data))

2177.0

## Split the Data

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state = 123)


In [31]:
X_train.shape

(1741, 100, 100, 3)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Instantiate the RandomForestClassifier
rf_model = RandomForestClassifier(
    criterion='gini',          # i. criterion: The function to measure the quality of a split. 'gini' for Gini impurity or 'entropy' for information gain.
    min_samples_split=2,        # ii. min_samples_split: The minimum number of samples required to split an internal node.
    min_samples_leaf=1,         # iii. min_samples_leaf: The minimum number of samples required to be at a leaf node.
    max_features=None,        # iv. max_features: The number of features to consider when looking for the best split. None means all features are considered.
    bootstrap=True,             # v. bootstrap: Whether to use bootstrapping (sampling with replacement) when building trees.
    random_state=42,             # vi. random_state: Seed used by the random number generator for reproducibility.
    n_jobs = -1
)

# Fit the model to the training data
nsamples, nx, ny, nrgb = X_train.shape
X_train2 = X_train.reshape((nsamples, nx * ny * nrgb))
rf_model.fit(X_train2, y_train)


RandomForestClassifier(max_features=None, n_jobs=-1, random_state=42)

In [ ]:
nsamples, nx, ny, nrgb = X_test.shape
x_test = X_test.reshape((nsamples, nx * ny * nrgb))

predictions = rf_model.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, predictions)
print(accuracy)

report = classification_report(y_test, predictions)
print(report)

0.44954128440366975
              precision    recall  f1-score   support

           0       1.00      0.32      0.48        57
           1       1.00      0.35      0.52        62
           2       0.91      0.34      0.50        29
           3       1.00      0.45      0.62        44
           4       1.00      0.50      0.67        28
           5       1.00      0.35      0.52        49
           6       0.85      0.44      0.58        63
           7       1.00      0.42      0.59        64

   micro avg       0.96      0.39      0.56       396
   macro avg       0.97      0.40      0.56       396
weighted avg       0.97      0.39      0.56       396
 samples avg       0.36      0.36      0.36       396



C:\Users\Icego\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Icego\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# It looks to me like the model is just a little timid, it's got very high precision, so when it's guesssing it is pretty accurate
# unfortunately, it didn't guess at all on 234 of the ~400 test cases
sum(sum(y_test)) - sum(sum(predictions))

234.0

## Convolutional Neural Network

### DataLoader

In [97]:
from torch import optim
from torch.utils.data import DataLoader

from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, X, Y, transform=None):
        self.X = X
        self.Y = Y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x_sample = self.X[idx]
        y_sample = self.Y[idx]

        if self.transform:
            x_sample = self.transform(x_sample)

        return x_sample, y_sample

In [98]:
from torchvision import transforms

img_transform = transforms.Compose([
    transforms.ToTensor(),  # Converts the image to a PyTorch tensor
])

In [99]:
train_ds = CustomDataset(X_train, y_train, img_transform)
val_ds = CustomDataset(X_test, y_test, img_transform)

In [100]:
train = DataLoader(train_ds, batch_size = 16)
val = DataLoader(val_ds, batch_size = 16)


In [101]:
# We have to transform images beccause they're are an extremely high dimensional space,
# if we transform it to a tensor it's just a 1 dim array which the computer likes

### Neural Network

In [120]:
from torch import nn

class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        
        self.cnn_layers = nn.Sequential(
            # in channels: # colours in input image (colour = 3)
            # out channels: # filters
            # kernel size: how big an area should we consider?
            # stride: # of pixels to move the kernel over by
            # padding: # of pixels to add at each edge of image (3x3 kernel : padding = 1 || 5x5 kernel: padding: 2)
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.linear_layers = nn.Sequential(nn.Linear(10000,9))
    
    def forward(self, X):
        output = self.cnn_layers(X)
        output = output.view(output.size(0), -1)
        output = self.linear_layers(output)
        return output
# Source: The Honorable Dr. Professor Mr. David Guy Brizan

In [121]:
# Layer 1: 3 input, 16 output channels (also known as filters), a kernel size of 3x3, a stride of 1 
#        (how much the filter moves each time), and padding of 1 (zero-padding to the input).
# Layer 2: This layer exists to Normalize the inputs to the next Conv layer, 
#           BatchNorm is meant to accelerate training and make results more consistent
# Layer 3: This activation layer breaks up the linear structure of outputs, and will squeeze outputs
#          All negative inputs will become 0
# Layer 4: MaxPool takes a 2x2 filter and returns only the max value for each square, this simplifies 
#          what we're working with, "Down-sampling"
# Layer 5: Goes from high dimensional inputs/ what we're working with in the network and squeezes it 
#          into the output array, length 9 with corresponding probabilites for each pose

In [122]:
# Why is the number of channels 3 for the first Convolutional layer? 
#    We have RGB images, so each channel will be responsible for one color
# Why is the first parameter 10,000 for the Linear layer?
#    Our images are 100x100, so the linear layer is responsible for taking that 
#    and squeezing it down to our output's size, a array of len 9

In [123]:
# Optimizers exist to nudge the model in the right direction for the most accurate predictions
# I like adamax, so I'll use that one 
# lr gives us a scalar multiple to reduce adjustments our optimizer gives so that we move slowly
#    and carefully towards the best possible model and avoid bouncing around never finding the best one. 

In [124]:

model = CNN().to(dtype=torch.float32)

optimiser = optim.Adamax(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss() #we're using CrossEntropyLoss as our optimizer
print(model)

CNN(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=10000, out_features=9, bias=True)
  )
)


In [138]:
epochs = 24

for epoch in range(epochs):
    total_loss = 0.
    for images, labels in train:
        optimiser.zero_grad()
        output = model(images.to(dtype=torch.float32))
        loss = criterion(output, labels)
        loss.backward()
        optimiser.step()
        total_loss += loss.item()
    print('Epoch', epoch, 'loss:', (total_loss / len(train)))

Epoch 0 loss: 0.05568740761693601
Epoch 1 loss: 0.06047326893921634
Epoch 2 loss: 0.05417832665982733
Epoch 3 loss: 0.057739354934919315
Epoch 4 loss: 0.052631065275075786
Epoch 5 loss: 0.05485115478716688
Epoch 6 loss: 0.04982763051627836
Epoch 7 loss: 0.051324222004041076
Epoch 8 loss: 0.047231105551414125
Epoch 9 loss: 0.048775000500902874
Epoch 10 loss: 0.044283013365769744
Epoch 11 loss: 0.049671571185906344
Epoch 12 loss: 0.04299164980803744
Epoch 13 loss: 0.04556016820811863
Epoch 14 loss: 0.04113416861606027
Epoch 15 loss: 0.04435534023577262
Epoch 16 loss: 0.03997006377957146
Epoch 17 loss: 0.042549457865630476
Epoch 18 loss: 0.039060305868096036
Epoch 19 loss: 0.04024390020907575
Epoch 20 loss: 0.03650630478119187
Epoch 21 loss: 0.037974757093769594
Epoch 22 loss: 0.034186421010275844
Epoch 23 loss: 0.036214417847377514


In [140]:
correct = 0
total = 0
for images, labels in val:
    optimiser.zero_grad()
    outputs = model(images.to(dtype=torch.float32))
    _, predicted = torch.max(outputs, 1)
    for i in range(len(labels)):
        pred = predicted[i]
        if labels[i][pred] == 1:
            correct += 1
            total +=1
        else:
            total+=1
        
print(f'{correct}/{total}, accuracy: {correct/total}')
# CNN accuracy is significantly better than the RandomForest, also runs way faster
# RFC accuracy was 0.44954128440366975

315/436, accuracy: 0.7224770642201835
